### NLP Setup

In [1]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from app.src.helpers.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

## Statements

In [101]:
s = 'the world is a vampire'

summarizer.summarize(s)


Sentence: the world is a vampire

i     TEXT            POS             TAG             DEP             LEMMA           HEAD            ENT            
--------------------------------------------------------------------------------------------------------------
0     the             DET             DT              det             the             world                          
1     world           NOUN            NN              nsubj           world           is                             
2     is              AUX             VBZ             ROOT            be              is                             
3     a               DET             DT              det             a               vampire                        
4     vampire         NOUN            NN              attr            vampire         is                             



               S                 
      _________|___               
     |             VP            
     |          ___|___           
     

c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [ ]:
# Can probably use a matcher here
# And also the noun chunks or noun phrases

# What were looking for: NP - V(linking) - NP/ADJ
## V(linking): Make a list of lemmas
## NP/ADJ

In [90]:
main_linking_verb_lemmas = ['be', 'become', 'seem']
sense_lemmas = ['look', 'appear', 'sound', 'taste', 'smell', 'feel']

lemma_list = main_linking_verb_lemmas + sense_lemmas

In [102]:
doc = nlp(s)

for nc in doc.noun_chunks:
    print(nc)

the world
a vampire


c:\Users\regan\Documents\dev\symboleo-nlp\sym-env\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [106]:
sent = list(doc.sents)[0]
print(sent._.parse_string)

cs = list(sent._.children)

if cs[0]._.labels[0] == 'NP':
    np = cs[0].text
    print('NP:', np)

if cs[1]._.labels[0] == 'VP':
    cs2 = list(cs[1]._.children)
    verb = cs2[0][0]

    if verb.pos_ in ['VERB', 'AUX']:
        if verb.lemma_ in lemma_list:
            verb_text = verb.text
            print('VERB:', verb_text)

    pred = cs2[1]
    # 
    pred_labels = list(pred._.labels)
    if len(pred_labels) > 0:
        if pred_labels[0] in ['ADJP', 'NP']:
            pred = pred.text
            print('PRED:', pred)



(S (NP (DT the) (NN world)) (VP (VBZ is) (NP (DT a) (NN vampire))))
NP: the world
VERB: is
PRED: a vampire


In [80]:
for x in sent._.children:
    print(x._.labels, x.text)
    for xx in x._.constituents:
        print('--', xx._.labels)
    # for xx in x._.children:
    #     print('--', xx._.labels)

('NP',) legal proceedings
-- ('NP',)
-- ()
-- ()
('VP',) become necessary
-- ('VP',)
-- ()
-- ('ADJP',)


In [ ]:
from spacy.matcher import PhraseMatcher, Matcher

matcher = Matcher(nlp.vocab)

main_linking_verb_lemmas = ['be', 'become', 'seem']
sense_lemmas = ['look', 'appear', 'sound', 'taste', 'smell', 'feel']
# other_linking_lemmas = ['grow, turn, remain, prove, act, stay, get']

# Cant use the matcher... We need to know about noun phrases
## Need something custom


linking_verb_patterns = [
    [
        { "" }
    ]
]

# separate ones for contract event, ob event, power event, etc?
domain_event_patterns = [
    [{"LOWER": {'IN': ['contract']}, "POS": "NOUN" } , {"POS": "VERB", "LEMMA": {"IN": contract_verbs}} ],
    [{"LOWER": {"IN": contract_event_nouns}, "POS": "NOUN"}, {"POS": {"IN": ["VERB", "AUX"]}, "OP": "+"}, {"LOWER": "not", "OP": "?"}, {"LEMMA": {"IN": contract_event_verbs}, "OP": "?"}],
    [{"POS": "NOUN", "LEMMA": {"IN": contract_verb_nouns}}, {"LOWER": "of"}, {"LOWER": "the"}, {"LOWER": {'IN': ['contract']}, "POS": "NOUN" }]
]

matcher.add("DOMAIN_EVENT", domain_event_patterns)

### Symboleo Generator


In [2]:
from app.src.helpers.template_getter import get_template

contract = get_template('sample_t')

In [3]:
results = []

for dk in contract.domain_model.events:
    domain_obj = contract.domain_model.events[dk]
    date_props = [x for x in domain_obj.props if x.type == 'Date']

    for dp in date_props:
        next_val = f'{domain_obj.name}.{dp.key}'
        results.append(next_val)

print(results)



['Delivered.delDueDate', 'Paid.payDueDate']


In [4]:
results = []

decs = contract.contract_spec.declarations

for dk in decs:
    print(dk)
    domain_obj = decs[dk]
    # date_props = [x for x in domain_obj.props if x.type == 'Date']

    # for dp in date_props:
    #     next_val = f'{domain_obj.name}.{dp.key}'
    #     results.append(next_val)

print(results)



goods
delivered
paidLate
paid
disclosed
[]


In [ ]:
symboleo_spec = contract_spec.to_sym()

In [ ]:
FILENAME = 'sample'
FILEPATH = f'./app/templates/sample/t/{FILENAME}.txt'

f = open(FILEPATH, 'w')
f.write(symboleo_spec)
f.close()

### Import the Template

In [ ]:
from app.templates.meat_sale.nl_template import nl_template

obs = nl_template['obligations']

print('OBLIGATIONS')
for ob in obs:
    print(f'{ob}: {obs[ob]}\n')

In [ ]:
from app.templates.meat_sale.symboleo import get_template

contract_template = get_template()

sym_template = contract_template.to_sym()

sym_template_path = './app/templates/meat_sale/symboleo/symboleo_spec.txt'
with open(sym_template_path, 'w') as f:
    f.write(sym_template)

In [ ]:
ex = 'deliver the goods within thirty days'

summarizer.summarize(ex)

## Graph

In [ ]:
from app.classes.spec.p_atoms import PAtomPredicate
from app.src.old.graph.graph_builder import GraphBuilder
from app.src.old.graph.graph_visualizer import GraphVisualizer
from app.classes.graph.digraph import Digraph

graph_builder = GraphBuilder()
graph_visualizer = GraphVisualizer()
digraph = graph_builder.build(PAtomPredicate)
gv = graph_visualizer.create_viz(digraph.nodes)
gv.show('nx.html')

## Workbench

In [ ]:
val = 'I arrived within 2 weeks of the project being completed.'

doc = nlp(val)

summarizer.summarize(val)

In [ ]:
# Identify the presence of the contract
from nltk.corpus import wordnet as wn 

In [ ]:
ss = wn.synsets('contract')

contract_synset = wn.synset('contract.n.01')

val = 'before the agreement terminates'
doc = nlp(val)

# Look for a noun that suggests presence of contract
## Noun chunk or Noun?
## What if it is qualified?
nouns = [x for x in doc if x.pos_ == 'NOUN']


noun_scores = []

for n in nouns:
    print(n.text)
    n_ss = wn.synsets(n.text, pos=wn.NOUN)
    
    nd = max([contract_synset.wup_similarity(ns) for ns in n_ss])
    noun_scores.append((n, nd))
    #print(n_ss)

top_ns = max(noun_scores, key=lambda x: x[1])
print(top_ns)

if top_ns[1] < 0.7:
    print('failed threshold')

# Look for dependence on event
## Case 1: 


# Contract event


# for syn in ss:
#     print(syn.name())
#     print(syn.definition())
#     print('\n')


Contract event
- presence of contract
- something happening to it - narrow set of verbs

examples
- termination of contract
- contract terminate
- the contract terminates

